In [1]:
!pip install torch torchvision matplotlib

In [3]:
import torch
import torch.nn as nn
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import time, os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


data_dir = "F:/PlantVillage"
train_dir = os.path.join(data_dir, "train")
val_dir = os.path.join(data_dir, "val")

In [23]:
# 图像预处理
data_transforms = {
    "train": transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor()
    ]),
    "val": transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])
}

# 加载数据
image_datasets = {
    "train": datasets.ImageFolder(train_dir, data_transforms["train"]),
    "val": datasets.ImageFolder(val_dir, data_transforms["val"])
}

dataloaders = {
    "train": DataLoader(image_datasets["train"], batch_size=32, shuffle=True),
    "val": DataLoader(image_datasets["val"], batch_size=32, shuffle=False)
}

class_names = image_datasets["train"].classes
print("Classes:", class_names)


Classes: ['Apple___Apple_scab', 'Apple___Black_rot', 'Apple___Cedar_apple_rust', 'Apple___healthy', 'Blueberry___healthy', 'Cherry_(including_sour)___Powdery_mildew', 'Cherry_(including_sour)___healthy', 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot', 'Corn_(maize)___Common_rust_', 'Corn_(maize)___Northern_Leaf_Blight', 'Corn_(maize)___healthy', 'Grape___Black_rot', 'Grape___Esca_(Black_Measles)', 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 'Grape___healthy', 'Orange___Haunglongbing_(Citrus_greening)', 'Peach___Bacterial_spot', 'Peach___healthy', 'Pepper,_bell___Bacterial_spot', 'Pepper,_bell___healthy', 'Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy', 'Raspberry___healthy', 'Soybean___healthy', 'Squash___Powdery_mildew', 'Strawberry___Leaf_scorch', 'Strawberry___healthy', 'Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___Late_blight', 'Tomato___Leaf_Mold', 'Tomato___Septoria_leaf_spot', 'Tomato___Spider_mites Two-spotted_spider_mite', 'Tomato___

In [7]:
model = models.mobilenet_v2(pretrained=True)

# 冻结特征提取层
for param in model.features.parameters():
    param.requires_grad = False

# 替换分类头（根据类别数量）
num_classes = len(class_names)
model.classifier[1] = nn.Linear(model.last_channel, num_classes)
model = model.to(device)

D:\anaconda\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
D:\anaconda\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to C:\Users\Nika/.cache\torch\hub\checkpoints\mobilenet_v2-b0353104.pth
100%|█████████████████████████████████████████████████████████████████████████████| 13.6M/13.6M [00:00<00:00, 21.1MB/s]


In [11]:
import os

train_dir = "F:/PlantVillage/train"  
total_images = 0

for root, dirs, files in os.walk(train_dir):
    image_files = [f for f in files if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    total_images += len(image_files)

print(f"Total training images: {total_images}")


Total training images: 44016


In [15]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.classifier.parameters(), lr=0.001)

num_epochs = 10

for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in dataloaders["train"]:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    acc = 100 * correct / total
    print(f"Train Loss: {running_loss:.4f}, Accuracy: {acc:.2f}%")


Epoch 1/10
Train Loss: 188.5012, Accuracy: 95.63%
Epoch 2/10
Train Loss: 176.3813, Accuracy: 95.71%
Epoch 3/10
Train Loss: 169.9178, Accuracy: 95.76%
Epoch 4/10
Train Loss: 164.9595, Accuracy: 95.93%
Epoch 5/10
Train Loss: 159.7064, Accuracy: 96.15%
Epoch 6/10
Train Loss: 154.5710, Accuracy: 96.29%
Epoch 7/10
Train Loss: 153.0787, Accuracy: 96.25%
Epoch 8/10
Train Loss: 149.6499, Accuracy: 96.34%
Epoch 9/10
Train Loss: 146.6369, Accuracy: 96.45%
Epoch 10/10
Train Loss: 147.5239, Accuracy: 96.44%


In [17]:
torch.save(model.state_dict(), "mobilenetv2_epoch10.pth")

In [7]:
from torchvision import models
import torch.nn as nn

# 重新创建 MobileNetV2 模型
model = models.mobilenet_v2(pretrained=False)

num_classes = 39  
model.classifier[1] = nn.Linear(model.last_channel, num_classes)

# 加载参数
model.load_state_dict(torch.load("mobilenetv2_epoch10.pth"))
model = model.to(device)
model.eval()
print("✅ Model loaded successfully.")


✅ Model loaded successfully.


In [13]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import os

# 路径设置（替换为你自己的验证集路径）
val_dir = "F:/PlantVillage/val"

# 定义验证集的图像预处理
val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

# 加载验证集
val_dataset = datasets.ImageFolder(val_dir, transform=val_transform)

# 创建 dataloader
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

dataloaders = {"val": val_loader}


In [17]:
criterion = nn.CrossEntropyLoss()  

model.eval()
val_loss = 0.0
correct = 0
total = 0

with torch.no_grad():
    for images, labels in dataloaders["val"]:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)

        loss = criterion(outputs, labels)       # ✅ 计算验证损失
        val_loss += loss.item()

        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

val_acc = 100 * correct / total
avg_val_loss = val_loss / len(dataloaders["val"])

print(f"✅ Validation Accuracy: {val_acc:.2f}%")
print(f"📉 Validation Loss: {avg_val_loss:.4f}")



✅ Validation Accuracy: 97.01%
📉 Validation Loss: 0.0898


In [21]:
#My training process went very smoothly, and the overall performance was excellent. From the training accuracy and loss, the model showed consistent convergence across all epochs. The training accuracy increased from 95.63% to 96.44%, while the loss steadily decreased, indicating that the optimizer and learning rate were well-tuned, and the model was effectively learning meaningful features.

#At the same time, the validation accuracy reached 97.01%, with a very low validation loss of just 0.0898. There was no sign of overfitting, as the validation performance did not lag behind the training performance—in fact, the model performed slightly better on the validation set. This suggests that the model has strong generalization ability, and the validation set may contain cleaner or more representative examples that match the learned features well.

#Given this training-validation behavior, I believe the number of training epochs was sufficient and that the model has already learned effectively. Further improvement might be possible by slightly increasing the number of epochs, applying light data augmentation, or exploring techniques like model distillation. However, based on the current results, I’m confident that the model is well-trained and ready for stable deployment.